In [5]:
import torch
import requests 
from bs4 import BeautifulSoup
import re
from transformers import AutoTokenizer,AutoModelForSequenceClassification

In [7]:
tokenizer=AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [8]:
model=AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [29]:
   tokens=tokenizer.encode('I hated this,absolutely the worst',return_tensors='pt')

In [30]:
tokens

tensor([[  101,   151, 39487, 10163, 10372,   117, 35925, 10563, 10103, 43060,
           102]])

In [31]:
tokenizer.decode(tokens[0])

'[CLS] i hated this, absolutely the worst [SEP]'

In [39]:
result=model(tokens)

In [40]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[ 4.8750,  1.7880, -0.8356, -3.0027, -2.0727]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [41]:
result.logits

tensor([[ 4.8750,  1.7880, -0.8356, -3.0027, -2.0727]],
       grad_fn=<AddmmBackward0>)

In [42]:
int(torch.argmax(result.logits))+1

1

In [43]:
tokens2=tokenizer.encode('this is amazing ',return_tensors='pt')

In [44]:
result2=model(tokens2)
result2.logits

tensor([[-1.9988, -2.3604, -0.8269,  1.1844,  3.2305]],
       grad_fn=<AddmmBackward0>)

In [45]:
int(torch.argmax(result2.logits))+1

5

In [51]:
r=requests.get('http://www.yelp.com/biz/mejico-sydney-2')
soup=BeautifulSoup(r.text,'html.parser')
regex=re.compile('.*comment*.')
results=soup.find_all('p',{'class':regex})
reviews=[result.text for result in results]

### to text of website
*r.text*

In [53]:
import pandas as pd 
import numpy as np

In [54]:
df=pd.DataFrame(np.array(reviews),columns=['reviews'])

In [55]:
df.head()

,reviews
0,Hotter than I imagine myself to be and firing ...
1,The food and service here was really good. It...
2,Visiting from Texas and decided to give this r...
3,"Great atmosphere, attentive service, solid mar..."
4,Don't come here expecting legit Mexican food b...


In [58]:
df['reviews'].iloc[0]

'Hotter than I imagine myself to be and firing up the flaming ring this place packs a firey heat punch with a delicious dolls mexican fusion. Pants free or margarita mash this is the place to explore.'

In [62]:
def sentiment_score(review):
    tokens=tokenizer.encode(review,return_tensors='pt')
    results=model(tokens)
    return int(torch.argmax(results.logits))+1

In [68]:
sentiment_score(df['reviews'].iloc[4])

3

In [69]:
sentiment_score(df['reviews'].iloc[0])

5

In [70]:
df['sentiment']=df['reviews'].apply(sentiment_score)

In [71]:
df['sentiment'].head()

0    5
1    5
2    5
3    3
4    3
Name: sentiment, dtype: int64

In [75]:
df

,reviews,sentiment
0,Hotter than I imagine myself to be and firing ...,5
1,The food and service here was really good. It...,5
2,Visiting from Texas and decided to give this r...,5
3,"Great atmosphere, attentive service, solid mar...",3
4,Don't come here expecting legit Mexican food b...,3
5,Out of all the restaurants that I tried in Syd...,5
6,We came here on a Thursday night @ 5pm and by ...,4
7,The food is fresh and tasty. The scallop cevi...,4
8,Have been here twice and have absolutely loved...,5
9,I was pleasantly surprised at what a great job...,5
